In [1]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import time as time_lib
from scipy.integrate import solve_ivp
from scripts.network import network
from scripts.training import trainModel
from scripts.dynamics import vecField,vecField_np
from scripts.parareal import parallel_solver
from scripts.ode_solvers import solver
from scripts.plotting import plot_results

In [2]:
torch.manual_seed(1)
np.random.seed(1)
dtype=torch.float32
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [3]:
vecRef = vecField_np()

beta,gamma,N = vecRef.beta,vecRef.gamma,vecRef.N
t_max = 100.
num_t = 101
L = 3
n_t = 2
vecRef.dt_fine = 1e-2
y0 = np.array([0.3,0.5,0.2])
d = 3

In [4]:
#Domain details
t0 = 0. #initial time
    
import pickle
with open('trainedModels/trained_model_SIR_20240424_161900.pkl', 'rb') as f:
    params = pickle.load(f)
    
dt = params["dt"]
nlayers = params["nlayers"]
hidden_nodes = params["hidden_nodes"]
act = params["act"]
dim_t = params["dim_t"]
device = params["device"]
file_name = params["file_name"]

model = network(neurons=hidden_nodes,d=d,dt=dt,act_name=act,nlayers=nlayers,dtype=dtype) #If you want a different network
model.to(device);

model.load_state_dict(torch.load(f"trainedModels/{file_name}.pt",map_location=device))
model.eval();

In [5]:
data = {"y0" : y0,
        "device":device,
        "dtype":dtype}

In [6]:
number_points = int(t_max / dt)
time = np.linspace(0,100,number_points+1)
dts = np.diff(time)

In [7]:
num_processors_list = [1]#np.arange(1,8)
computational_times = []
overheads = []
used_processors = []
for number_processors in num_processors_list:
    print(f"Experiment with {number_processors} processors")
    coarse_values_parareal,networks, total_time, number_processors, overhead_costs = parallel_solver(time,data,dts,vecRef,number_processors,model,verbose=True)
    used_processors.append(number_processors)
    overheads.append(overhead_costs)
    computational_times.append(total_time)

Experiment with 1 processors
Average cost first iterate :  0.0008354306221008301


Time for iterate 1 is 0.08354306221008301




Time required for the fine solver at iterate 2:  0.33270812034606934 


Solution time for the fine integrator : 0.3318827152252197
Overhead cost : 0.0008254051208496094


Norms differences :  [0.00000000e+00 3.22031113e-05 5.57887551e-05 7.29290304e-05
 8.81900283e-05 1.04720850e-04 1.22968584e-04 1.41346979e-04
 1.57921402e-04 1.71371971e-04 1.81116707e-04 1.87341673e-04
 1.91028029e-04 1.93431546e-04 1.96506190e-04 2.02083519e-04
 2.11616545e-04 2.25820630e-04 2.44570174e-04 2.67217979e-04
 2.92774710e-04 3.20092665e-04 3.48218622e-04 3.76272130e-04
 4.03581530e-04 4.29649919e-04 4.54092298e-04 4.76669553e-04
 4.97191232e-04 5.15546315e-04 5.31736501e-04 5.45755461e-04
 5.57654940e-04 5.67453413e-04 5.75205098e-04 5.80978067e-04
 5.84869711e-04 5.86972149e-04 5.87290660e-04 5.85915818e-04
 5.82918242e-04 5.78314557e-04 5.72213912e-04 5.64616186e-04
 5.55633054e-04 5.45379917e-04 5.33848926e-04 5.21158349e-04
 5.07486923e-04 4.93064870e-04 4.77914827e-04 4.62319060e-04
 4.46514576e-

In [8]:
def get_detailed_solution():
    num_steps = int((dts[0] / vecRef.dt_fine))
    time_plot = np.linspace(0,dts[0],num_steps)
    sol = model.plotOverTimeRange(y0,time_plot,data)
    total_time = time_plot
    for i in np.arange(len(dts)):
        num_steps = int((dts[i] / vecRef.dt_fine))
        time_plot = np.linspace(0,dts[i],num_steps)[1:]
        sol = np.concatenate((sol,model.plotOverTimeRange(coarse_values_parareal[i+1],time_plot,data)),axis=1)
        total_time = np.concatenate((total_time,time_plot+total_time[-1]),axis=0)
    return sol,total_time

In [9]:
network_sol, time_plot = get_detailed_solution()

In [10]:
initial = time_lib.time()
arg = [[y0,time_plot[-1],time_plot],vecRef]
output,time_plot_sequential = solver(arg,final=False)
#output, _ = dop853(funcptr, y0, t_eval=time_plot, rtol=1e-11, atol=1e-10)
final = time_lib.time()
print(f"Computational time sequential: {final-initial}")
print(f"Computational time parallel with {number_processors} processors: {total_time}")

Computational time sequential: 0.26129913330078125
Computational time parallel with 1 processors: 0.9884881973266602


In [11]:
if len(y0)==2:
        list_of_labels = [r"$\mathbf{x}_1$",r"$\mathbf{x}_2$"]
elif len(y0)==3:
        list_of_labels = [r"$\mathbf{x}_1$",r"$\mathbf{x}_2$",r"$\mathbf{x}_3$"]
elif len(y0)==4:
        list_of_labels = [r"$\mathbf{x}_1$",r"$\mathbf{x}_1'$",r"$\mathbf{x}_2$",r"$\mathbf{x}_2'$"]
else:
        list_of_labels = []
plot_results(y0,time_plot,time_plot_sequential,output,network_sol,list_of_labels,total_time,time)